In [0]:
#*********EDIT:- I apologise for misinterpreting validation accuracy as training accuracy in the Internshala questionnaire
# I have achieved validation accuracy of 98% with 7570 parameters*******************
# I have implemented early stopping in this notebook.
# If we continue training for upto 50 epochs, the training accuracy reaches 99.5% while the validation error is lesser indicating overfitting.

In [0]:
%tensorflow_version 1.x

TensorFlow 1.x selected.


In [0]:
import keras
from keras.models import Sequential
from keras.utils import np_utils
from keras.layers import Dense, Activation, Flatten, Dropout, BatchNormalization
from keras.layers import Conv2D, MaxPooling2D
from keras.callbacks import EarlyStopping
from keras.preprocessing.image import ImageDataGenerator
from keras.datasets import mnist
from keras import regularizers
from sklearn.model_selection import train_test_split
import numpy as np
random_seed = 2
np.random.seed(random_seed)

Using TensorFlow backend.


In [0]:
(X_train, y_train), (X_test, y_test) = mnist.load_data() #load the data 

In [0]:
import matplotlib.pyplot as plt
from google.colab.patches import cv2_imshow
import cv2
cv2_imshow(X_train[0]) #show the first image in the training dataset

In [0]:
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')

In [0]:
#resizing
X_train = X_train.reshape(-1, 28, 28, 1)
X_test = X_test.reshape(-1, 28, 28, 1)

In [0]:
#normalization
X_train = X_train/255;
X_test = X_test/255;

In [0]:
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size = 0.1, random_state=random_seed)

In [0]:
#build the model
model = Sequential()
model.add(Conv2D(6, kernel_size=(7,7), input_shape=(28,28,1), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(10,activation='softmax'))

In [0]:
model.summary() #details of the model

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_3 (Conv2D)            (None, 22, 22, 6)         300       
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 11, 11, 6)         0         
_________________________________________________________________
flatten_3 (Flatten)          (None, 726)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 10)                7270      
Total params: 7,570
Trainable params: 7,570
Non-trainable params: 0
_________________________________________________________________


In [0]:
es = EarlyStopping(monitor='val_acc', mode='max', verbose=1)

In [0]:
model.compile(optimizer='adam', 
              loss='sparse_categorical_crossentropy', 
              metrics=['accuracy'])

In [0]:
model.fit(x=X_train,y=y_train, validation_data = (X_val,y_val), epochs=50, callbacks=[es])
# If we add more parameters the accuracy will be much better
# Early stopping only allows 6 epochs
# Validation accuracy is nearly 98%

Train on 54000 samples, validate on 6000 samples
Epoch 1/50
54000/54000 [==============================] - 5s 98us/step - loss: 0.3269 - acc: 0.9061 - val_loss: 0.1599 - val_acc: 0.9518
Epoch 2/50
54000/54000 [==============================] - 5s 92us/step - loss: 0.1146 - acc: 0.9668 - val_loss: 0.1080 - val_acc: 0.9688
Epoch 3/50
54000/54000 [==============================] - 5s 93us/step - loss: 0.0822 - acc: 0.9761 - val_loss: 0.0788 - val_acc: 0.9757
Epoch 4/50
54000/54000 [==============================] - 5s 92us/step - loss: 0.0697 - acc: 0.9791 - val_loss: 0.0754 - val_acc: 0.9783
Epoch 5/50
54000/54000 [==============================] - 5s 92us/step - loss: 0.0602 - acc: 0.9814 - val_loss: 0.0673 - val_acc: 0.9798
Epoch 6/50
54000/54000 [==============================] - 5s 95us/step - loss: 0.0547 - acc: 0.9835 - val_loss: 0.0655 - val_acc: 0.9792
Epoch 00006: early stopping


In [0]:
#try data augmentation
datagen = ImageDataGenerator(
        zca_whitening=False, 
        rotation_range=10,  
        zoom_range = 0.1, 
        width_shift_range=0.1,  
        height_shift_range=0.1, 
        horizontal_flip=False, 
        vertical_flip=False) 
datagen.fit(X_train)

In [0]:
history = model.fit_generator(datagen.flow(X_train,y_train, batch_size=32),
                              epochs = 50, validation_data = (X_val,y_val),
                              verbose = 2, steps_per_epoch=X_train.shape[0] // 32,
                               callbacks=[es])
#even data augmentation gives about 97.5% validation accuracy

Epoch 1/50
 - 16s - loss: 0.5861 - acc: 0.8218 - val_loss: 0.1620 - val_acc: 0.9527
Epoch 2/50
 - 16s - loss: 0.2831 - acc: 0.9165 - val_loss: 0.1226 - val_acc: 0.9678
Epoch 3/50
 - 16s - loss: 0.2304 - acc: 0.9329 - val_loss: 0.1029 - val_acc: 0.9680
Epoch 4/50
 - 15s - loss: 0.2082 - acc: 0.9387 - val_loss: 0.1029 - val_acc: 0.9683
Epoch 5/50
 - 16s - loss: 0.1906 - acc: 0.9441 - val_loss: 0.0989 - val_acc: 0.9705
Epoch 6/50
 - 16s - loss: 0.1816 - acc: 0.9461 - val_loss: 0.0869 - val_acc: 0.9733
Epoch 7/50
 - 16s - loss: 0.1745 - acc: 0.9493 - val_loss: 0.0814 - val_acc: 0.9750
Epoch 8/50
 - 16s - loss: 0.1671 - acc: 0.9507 - val_loss: 0.0831 - val_acc: 0.9742
Epoch 00008: early stopping
